In [1]:
import os

# Load the information of the experiments with default performance and no form of balancing or 

# Check confusion matrices of the best performing models

After training without any form of balancing or weighing, the best performing modes were RandomForest, LinearSVC, SVC, and the RidgeClassifier.

Let's look at them closely, particularly at their confusion matrice